<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Langgraph_Agentic_RAG_Cyber_AI_Copilot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Cyber AI Copilot for Security and Intelligence Domain**

In [1]:
!pip install --upgrade --quiet sentence-transformers langchain langchain-groq langchain-pinecone langchain-core asknews langgraph
%pip install --quiet -U "langchain-community>=0.2.16"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.7/108.7 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8

In [2]:
import os
from typing import List, Dict, Any, Optional, TypedDict
from pydantic import BaseModel
from langchain_groq import ChatGroq
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langgraph.graph import StateGraph, END
from langchain_community.tools.asknews import AskNewsSearch
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.tools import JinaSearch
from dotenv import load_dotenv
from datetime import datetime, timedelta

# Load environment variables
load_dotenv()

# API Keys (hidden for security purposes)
GROQ_API_KEY = "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"
PINECONE_API_KEY = "8e15b925-3b96-497d-b20a-08d308782b83"
PINECONE_ENVIRONMENT = "us-east-1"
ASKNEWS_CLIENT_ID = "a0de4609-b760-4c83-9609-5c04d7743b84"
ASKNEWS_CLIENT_SECRET = "D5Mlhkztk4TcW24diUgcW0FA2w"
SERPER_API_KEY = "d8e815ef6caa94dbef7b977a0ea7d505b43a5a06"

# Set environment variables for AskNews
os.environ["ASKNEWS_CLIENT_ID"] = ASKNEWS_CLIENT_ID
os.environ["ASKNEWS_CLIENT_SECRET"] = ASKNEWS_CLIENT_SECRET
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

In [8]:
# Initialize the LLM and embeddings
llm = ChatGroq(temperature=0, model="llama-3.1-8b-instant", api_key=GROQ_API_KEY)
embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

# Initialize Pinecone and vector store
pc = Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)
pinecone_index = pc.Index("new-cyber-search")
vector_store = PineconeVectorStore(index=pinecone_index, embedding=embeddings)

# Initialize search tool
asknews_tool = AskNewsSearch(max_results=5)
google_serper = GoogleSerperAPIWrapper()
jina_search = JinaSearch()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
class AgentState(TypedDict):
    messages: List[Dict[str, str]]
    memory: Optional[Dict[str, Any]]

class SearchResult(BaseModel):
    source: str
    title: str
    snippet: str
    url: str
    date: Optional[str]

def vector_search(query: str) -> List[SearchResult]:
    results = vector_store.similarity_search(query, k=5)
    return [
        SearchResult(
            source="Vector Search",
            title=f"Result {i+1}",
            snippet=doc.page_content,
            url=doc.metadata.get("source", "No URL"),
            date=doc.metadata.get("date")
        ) for i, doc in enumerate(results)
    ]

def asknews_search(query: str) -> List[SearchResult]:
    results = asknews_tool.invoke({"query": query})
    return [
        SearchResult(
            source="AskNews",
            title=result.get("title", "No title"),
            snippet=result.get("snippet", "No snippet"),
            url=result.get("link", "No link"),
            date=result.get("date")
        ) for result in results
    ]

def google_serper_search(query: str) -> List[SearchResult]:
    results = google_serper.results(query)
    return [
        SearchResult(
            source="Google Serper",
            title=result.get("title", "No title"),
            snippet=result.get("snippet", "No snippet"),
            url=result.get("link", "No link"),
            date=result.get("date")
        ) for result in results.get("organic", [])
    ]

def jina_search(query: str) -> List[SearchResult]:
    results = jina_search.invoke({"query": query})
    return [
        SearchResult(
            source="Jina Search",
            title=result.get("title", "No title"),
            snippet=result.get("snippet", "No snippet"),
            url=result.get("link", "No link"),
            date=result.get("date")
        ) for result in results
    ]

In [4]:
def execute_searches(state: AgentState) -> AgentState:
    query = state["messages"][-1]["content"]
    searches = [
        ("Vector Search", vector_search),
        ("AskNews Search", asknews_search),
        ("Google Serper Search", google_serper_search),
        ("Jina Search", jina_search)
    ]

    all_results = []
    for name, func in searches:
        try:
            results = func(query)
            all_results.extend(results)
        except Exception as e:
            state["messages"].append({"role": "tool", "content": f"{name} Error: {str(e)}"})

    # Sort results by date (if available) and relevance
    all_results.sort(key=lambda x: (x.date is not None, x.date or "", x.title), reverse=True)

    # Select top 10 most relevant results
    top_results = all_results[:10]

    state["messages"].append({"role": "tool", "content": "Search Results", "results": top_results})
    return state

def format_source_link(source: str, url: str) -> str:
    return f"[{source}]({url})"

In [6]:
import re
def generate_response(state: AgentState) -> AgentState:
    memory = state.get("memory", {})
    chat_history = memory.get("chat_history", "")

    search_results = next((m["results"] for m in reversed(state["messages"]) if m["role"] == "tool" and "results" in m), [])

    prompt = ChatPromptTemplate.from_messages([(
        "system", """You are an advanced AI specializing in cybersecurity and intelligence analysis. Your task is to provide highly relevant, actionable, and up-to-date insights based on the latest cyber incidents. Follow these guidelines:

1. Analyze all search results thoroughly, prioritizing the most recent and relevant information.
2. Focus on information from reputable sources, official reports, and verified cybersecurity platforms.
3. Identify emerging patterns, trends, and potential implications of the latest cyber incidents.
4. Provide a structured response with the following sections:
   a. Key Highlights (bullet points of the most critical findings)
   b. Trends & Implications (numbered list of important trends and their potential impact)
   c. Critical Action Items (prioritized list of recommended actions)
   d. Emerging Concerns (potential future threats or evolving risks)
   e. Research Gaps (areas that require further investigation)

5. Include clear citations for all information, using the format [Source Name](URL).
6. Prioritize information from the last 30 days. If using older sources, clearly state the date.
7. If critical information is missing, identify gaps and suggest additional areas for investigation.
8. Maintain a concise yet comprehensive format, ensuring all key points are covered within a 750-word limit.
9. Use Markdown formatting for emphasis: **bold** for section headers, and - for bullet points.

Previous conversation: {chat_history}
Human query: {input}
Search Results: {search_results}

Provide a structured, actionable response based on the latest findings:
"""
    )])

    chain = prompt | llm

    response = chain.invoke({
        "input": state["messages"][-1]["content"],
        "search_results": "\n".join([f"{result.title}\n{result.snippet}\n{format_source_link(result.source, result.url)}\n" for result in search_results]),
        "chat_history": chat_history
    })

    # Post-process the response to add highlights
    processed_response = add_highlights(response.content)

    state["messages"].append({"role": "assistant", "content": processed_response})
    state["memory"] = {"chat_history": chat_history + f"\nHuman: {state['messages'][-2]['content']}\nAI: {processed_response}"}
    return state

def add_highlights(text: str) -> str:
    # Add yellow highlight to key phrases
    highlight_phrases = [
        "Critical vulnerability",
        "Zero-day exploit",
        "Ransomware attack",
        "Data breach",
        "Advanced Persistent Threat",
        "Supply chain attack",
        "Phishing campaign",
        "Malware outbreak",
        "Cybersecurity best practice",
        "Emerging threat"
    ]

    for phrase in highlight_phrases:
        text = re.sub(f"({phrase})", r"<span style='background-color: #FFFF00'>\1</span>", text, flags=re.IGNORECASE)

    return text

# Workflow definition
workflow = StateGraph(AgentState)
workflow.add_node("execute_searches", execute_searches)
workflow.add_node("generate_response", generate_response)
workflow.add_edge("execute_searches", "generate_response")
workflow.add_edge("generate_response", END)
workflow.set_entry_point("execute_searches")
graph = workflow.compile()

def run_agent(query: str, memory: Optional[Dict[str, Any]] = None) -> AgentState:
    state = AgentState(messages=[{"role": "human", "content": query}], memory=memory or {})
    return graph.invoke(state)

In [9]:
if __name__ == "__main__":
    query = "Latest Cyber Incidents from Blackbasta Ransomware?"
    result = run_agent(query)
    for message in result["messages"]:
        if message["role"] == "assistant":
            print("AI Copilot Analysis:")
            print(message["content"])

<ipython-input-4-a0f0b2abb74b>:13: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = func(query)


AttributeError: 'str' object has no attribute 'date'